#Парсим документ

In [52]:
"""
    Для парсинга используем библиотеку csv
    csv.DictReader() - создает словарь из объекта *.csv ключи в котором это название столбцов (в оригинале)
    метод fieldnames - выводит имена ключей в списке 
    strip() возвращает копию строки, в которой все символы были удалены с начала и конца строки (символом по умолчанию является пробел).
"""

import csv

with open('example.csv', 'r') as f:
    reader = csv.DictReader(f)
#     print(reader.fieldnames)
    #Выводим содержимое всех столбцов в соответсвии с их ключами 
    lines = list(reader)
#     print(lines[35]["Код BOT_CONFIG"])
    
all_bot_config_dict = [key['Код BOT_CONFIG'] for key in lines]
# print(all_bot_config_dict)

In [55]:
config_codes = []

for code in all_bot_config_dict:
    #Обрезаем из строки только содержимое ключа BOT_CONFIG = {...}
    if 'BOT_CONFIG =' in code:
        code = code.split('BOT_CONFIG =')[1]
    if 'def ' in code:
        code = code.split('\ndef ')[0]
    #Удаляем все символы пробела в начале и в конце строки
    code = code.strip()
    config_codes.append(code)

In [62]:
#eval- позволить считать словарь выражением и вставит его как элемент списка
# try:
#     response = eval("{'a': 'b'}")
#     print(response)
# except:
#     print(1)
configs = []
errors = 0
for code in config_codes:
    # Добавляем словарь с intentom в список, если у нас правильный словарь то функция eval обработает его как значение и поместит в список
    #     print(code)
    try:
        configs.append(eval(code))
    except Exception as e:
        errors += 1


print(len(configs))
print(errors)

52
23


In [85]:
big_config = {
    'intents': {},
    'failure_phrases': []
}
# print(configs)
#Переберием список словарей намерений 
for config in configs:
#     print(type(config),len(config))
    #Если элемент конфиг -словарь, проверяем что есть ключ intents, если да -то перебираем его по ключу и значению
    if isinstance(config, dict):
        if 'intents' in config:
            for intent, value in config['intents'].items():
#                 print(intent)
#                 print('\t', value)
                #Если текущего намерения нет в отфильрованном списке, то добавляем новый словарь намерения с вопросами и ответами
                if intent not in big_config['intents']:
                    big_config['intents'][intent] = {
                        'examples': [],
                        'responses': []
                    }
                #Если вопросы(и ответы) это словари , то добавляем в наше новое намерение зачения в ключи examples/responses
                if isinstance(value, dict):
                    big_config['intents'][intent]['examples'] += value.get('examples', [])
                    big_config['intents'][intent]['responses'] += value.get('responses', [])
        #Добавляем из пользовательских списков намерений в наш обновленный список фразы заглушки 
        big_config['failure_phrases'] += config.get('failure_phrases', [])

In [88]:
for intent, value in big_config['intents'].items():
    value['examples'] = list(set(value['examples']))
    value['responses'] = list(set(value['responses']))
    value['examples'] = [s.strip() for s in value['examples'] if s.strip()]
    value['responses'] = [s.strip() for s in value['responses'] if s.strip()]

big_config['failure_phrases'] = list(set(big_config['failure_phrases']))
big_config['failure_phrases'] = [s.strip() for s in big_config['failure_phrases'] if s.strip()]

In [90]:
# big_config

In [95]:
# print(big_config)
print(type(big_config))

<class 'dict'>


Подготовка датасета

In [101]:
BIG_CONFIG = big_config
#Создаем список из данных формата [значение из examples: intent к которому относится значение] дял дальнейшей вектоиизации
data_set = []
for intent, intent_data in BIG_CONFIG['intents'].items():
    for exmple_answer in intent_data['examples']:
        data_set.append([exmple_answer,intent])

# print(data_set)

In [102]:
#Создаем отдельно множество ответов
X_text = [x for x, y in data_set]
y = [y for x,y in data_set]

In [108]:
# print(len(X_text))

Векторизация

In [121]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_text)

Классификация

In [124]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [128]:
clf.predict(vectorizer.transform(['Сколько время?']))

array(['time'], dtype='<U28')

Подсчет качества

In [129]:
clf.score(X,y)

0.6289424860853432

In [130]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [131]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [132]:
clf.score(X_test, y_test)

0.21910112359550563

In [136]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC

In [137]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))  # Как улучшить?
X = vectorizer.fit_transform(X_text)

probas = []

for i in range(10):
    clf = LinearSVC()  # Как улучшить?

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    clf.fit(X_train, y_train)
    proba = clf.score(X_test, y_test)
    probas.append(proba)

print(sum(probas) / len(probas))

0.3205056179775281
